# Contrastive Pretraining for Time series 

In [5]:
import os 
import copy
os.system("unset LD_LIBRARY_PATH")
import numpy as np
from itertools import product

import datetime
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader    

# Dataset classes and functions

In [6]:
class DatasetConfig():
    def __init__(self):
        self.horizon = 48
        self.start_time = 0
        self.end_time = 2*np.pi 

        self.minimum_amplitude = 0.1
        self.maximum_amplitude = 1.0
        self.amplitude_interval = 0.05
        self.amplitudes = list(np.arange(self.minimum_amplitude, self.maximum_amplitude + self.amplitude_interval, self.amplitude_interval))
    
        self.minimum_frequency = 0.1
        self.maximum_frequency = 1 
        self.frequency_interval = 0.05
        self.frequencies = list(np.arange(self.minimum_frequency, self.maximum_frequency + self.frequency_interval, self.frequency_interval))

        self.minimum_phase = 0.0 
        self.maximum_phase = 2*np.pi
        self.phase_interval = 5*np.pi/180
        self.phases = list(np.arange(self.minimum_phase, self.maximum_phase, self.phase_interval))

        self.sine_params = list(product(self.amplitudes, self.frequencies, self.phases))

        self.train_test_ratio = 0.95
        

class Sinusoidal():
    def __init__(self, config):
        self.config = config

        time_stamps = np.linspace(self.config.start_time, self.config.end_time, self.config.horizon)
        data = [params[0]*np.sin(2*np.pi*params[1]*time_stamps + params[2]) for params in self.config.sine_params]
        
        self.data = np.array(data)
        self.labels = np.array([np.array(params) for params in self.config.sine_params])

        self.data = torch.tensor(self.data).type(torch.FloatTensor)
        self.data = self.data.unsqueeze(-1)
        self.labels = torch.tensor(self.labels).type(torch.FloatTensor)

        indices = torch.randperm(self.data.size(0))
        num_training_indices = int(self.data.size(0)*self.config.train_test_ratio)
        train_indices = indices[:num_training_indices]
        test_indices = indices[num_training_indices:]
        self.train_data = self.data[train_indices]
        self.train_labels = self.labels[train_indices]
        self.test_data = self.data[test_indices]
        self.test_labels = self.labels[test_indices]

    
class GetDataset(Dataset):
    def __init__(self, data, labels):
        self.dataset = data
        self.labels = labels
    
    def __getitem__(self, org_index):
        return {"time_series": self.dataset[org_index], "labels": self.labels[org_index]}

    def __len__(self):
        return self.dataset.shape[0]

# Model architecture

In [7]:
class CLTSPConfig():
    def __init__(self):
        # train config parameters
        self.batch_size = 64
        self.n_epochs = 2000
        self.learning_rate = 1e-5
        self.n_plots = 6

        # model parameters
        self.n_features = 1
        self.n_params = 3
        self.embedding_dim = 128

        # model initialization 
        self.pretrained_loc = 'save/sine_20230810_151623/results/weights/best_model.pth'

class TSEncoder(nn.Module):
    def __init__(self, seq_len, n_features, hidden_layer_output_dim=16, num_layers=4, bidirectional=True):
        super(TSEncoder, self).__init__()

        self.seq_len, self.n_features = seq_len, n_features
        self.hidden_layer_output_dim = hidden_layer_output_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.flat_dim = self.num_layers*2*self.hidden_layer_output_dim if self.bidirectional else self.num_layers*self.hidden_layer_output_dim

        self.rnn1 = nn.LSTM(
            input_size=n_features, # 1
            hidden_size=32, # 32
            num_layers=self.num_layers,
            batch_first=True,
            bidirectional=self.bidirectional
        )
        
        self.mean = nn.LSTM(
            input_size=32*2, 
            hidden_size=self.hidden_layer_output_dim,
            num_layers=self.num_layers,
            batch_first=True,
            bidirectional=True
        )

        self.var = nn.LSTM(
            input_size=32*2, 
            hidden_size=self.hidden_layer_output_dim,
            num_layers=self.num_layers,
            batch_first=True,
            bidirectional=self.bidirectional
        )

    def forward(self, x):
        x, (_, _) = self.rnn1(x) # the shape of x is 1x140x32
        _, (mu, _) = self.mean(x)
        _, (sigma, _) = self.var(x)
        mu = mu.permute(1,0,2).reshape(-1, self.flat_dim)
        sigma = sigma.permute(1,0,2).reshape(-1, self.flat_dim)
        return mu, sigma

class TSDecoder(nn.Module):
    def __init__(self, seq_len, hidden_layer_output_dim=64, n_features=1, num_layers=4, bidirectional=True):
        super(TSDecoder, self).__init__()

        self.num_layers = num_layers
        self.bidirectional = bidirectional 
        self.input_size = self.num_layers*2*hidden_layer_output_dim if self.bidirectional else self.num_layers*hidden_layer_output_dim
        self.seq_len = seq_len
        self.n_features = n_features

        self.rnn1 = nn.LSTM(
            input_size=self.input_size,
            hidden_size=32,
            num_layers=1,
            batch_first=True,
            bidirectional=self.bidirectional
        )

        self.rnn2 = nn.LSTM(
            input_size=32*2,
            hidden_size=hidden_layer_output_dim,
            num_layers=1,
            batch_first=True,
            bidirectional=self.bidirectional
        )

        self.rnn3 = nn.LSTM(
            input_size=hidden_layer_output_dim*2,
            hidden_size=1,
            num_layers=1,
            batch_first=True,
            bidirectional=False
        )

    def forward(self, x):
        B = x.shape[0]
        x = x.unsqueeze(1)
        x = x.repeat(1, self.seq_len, 1)
        x, (_, _) = self.rnn1(x)
        x, (_, _) = self.rnn2(x)
        x, (_, _) = self.rnn3(x)
        return x

class ParamEncoder(nn.Module):
    def __init__(self, input_size, output_size):
        super(ParamEncoder, self).__init__()

        self.fc1 = nn.Linear(input_size, int(output_size // 2))
        self.act1 = nn.ReLU()
        self.fc2 = nn.Linear(int(output_size // 2), output_size)
        self.act2 = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

  
    def forward(self, x):
        x = self.dropout(self.act1(self.fc1(x)))
        x = self.dropout(self.act2(self.fc2(x)))
        return x

class CLTSPRecurrentAutoencoder(nn.Module):
    def __init__(self, seq_len, n_features, n_params, embedding_dim=64):
        super(CLTSPRecurrentAutoencoder, self).__init__()

        self.num_layers = 4
        self.bidirectional = True 
        self.embedding_dim = embedding_dim
        self.hidden_layer_output_dim = int(embedding_dim / (self.num_layers*2)) if self.bidirectional else int(embedding_dim / self.num_layers)
        
        self.ts_encoder = TSEncoder(seq_len, n_features, self.hidden_layer_output_dim, self.num_layers, self.bidirectional)
        self.ts_decoder = TSDecoder(seq_len, self.hidden_layer_output_dim, n_features, self.num_layers, self.bidirectional)
        self.param_encoder = ParamEncoder(input_size=n_params, output_size=embedding_dim)


    def reparameterization(self, mean, var):
        epsilon = torch.randn_like(var)        
        z = mean + var*epsilon
        return z

    def forward(self, timeseries, parameters):
        mu, log_var = self.ts_encoder(timeseries)
        z = self.reparameterization(mu, torch.exp(0.5 * log_var))
        reconstruced = self.ts_decoder(z)
        
        label_projection = self.param_encoder(parameters)
        timeseries_projection = z

        return reconstruced, mu, log_var, label_projection, timeseries_projection

# Dataset and Model instantiation

In [8]:
dataset_config = DatasetConfig() 
cltsp_config = CLTSPConfig()

sine_dataset_obj = Sinusoidal(config=dataset_config)
train_dataset = GetDataset(data=sine_dataset_obj.train_data, labels=sine_dataset_obj.train_labels)
test_dataset = GetDataset(data=sine_dataset_obj.test_data, labels=sine_dataset_obj.test_labels)

train_dataloader = DataLoader(train_dataset, batch_size=cltsp_config.batch_size, num_workers=0, shuffle=True)
val_dataloader = DataLoader(test_dataset, batch_size=cltsp_config.batch_size, num_workers=0, shuffle=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CLTSPRecurrentAutoencoder(seq_len=dataset_config.horizon, n_features=cltsp_config.n_features, n_params=cltsp_config.n_params, embedding_dim=cltsp_config.embedding_dim)
model = model.to(device)
if cltsp_config.pretrained_loc != '':
    model.load_state_dict(torch.load(cltsp_config.pretrained_loc))


# Training and Logging

In [99]:
def plot_results(model, val_dataloader, num_plots, log_dir, epoch, device):
    fig, axs = plt.subplots(
        nrows=1,
        ncols=num_plots,
        sharey=True,
        sharex=True,
        figsize=(32, 8)
    )

    model = model.eval()
    for val_batch in val_dataloader:
        seq_true = val_batch["time_series"]
        seq_true_labels = val_batch["labels"]

        seq_true = seq_true.to(device)
        seq_true_labels = seq_true_labels.to(device)
        predictions, _, _, _, _ = model(seq_true, seq_true_labels)
        for i in range(num_plots):
            pred = predictions[i].squeeze(-1).cpu().detach().numpy()
            true = seq_true[i].squeeze(-1).cpu().numpy()
            axs[i].plot(true, label='true')
            axs[i].plot(pred, label='reconstructed')
        break
    fig.tight_layout()

    save_dir = os.path.join(log_dir, 'qualitative')
    os.makedirs(save_dir, exist_ok=True)
    save_loc = os.path.join(save_dir, str(epoch)+'.png')
    plt.savefig(save_loc)
    plt.close('all')

def cross_entropy(preds, targets, reduction='none'):
    log_softmax = nn.LogSoftmax(dim=-1)
    loss = (-targets * log_softmax(preds)).sum(1)
    if reduction == "none":
        return loss
    elif reduction == "mean":
        return loss.mean()

def loss_function(x, x_hat, mean, log_var, label_projection, timeseries_projection, criterion):
    reconstruction_loss = criterion(x_hat, x) 
    kl_divergence_loss = - 0.5 * torch.mean(1+ log_var - mean.pow(2) - log_var.exp())
    # print(reproduction_loss, KLD)

    # Calculating the Loss
    logits = (label_projection @ timeseries_projection.T)
    labels_similarity = label_projection @ label_projection.T
    timeseries_similarity = timeseries_projection @ timeseries_projection.T
    targets = F.softmax((labels_similarity + timeseries_similarity) / 2, dim=-1)
    labels_loss = cross_entropy(logits, targets, reduction='none')
    timeseries_loss = cross_entropy(logits.T, targets.T, reduction='none')
    similarity_loss =  (labels_loss + timeseries_loss) / 2.0
    similarity_loss = similarity_loss.mean()

    return reconstruction_loss, kl_divergence_loss, similarity_loss

    

def train_model(model, train_dataloader, val_dataloader, train_config, log_dir, device):
    optimizer = torch.optim.Adam(model.parameters(), lr=train_config.learning_rate)
    criterion = nn.L1Loss(reduction='sum').to(device)
    history = dict(train=[], val=[])

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 10000000000.0
    n_epochs = train_config.n_epochs

    for epoch in range(n_epochs + 1):
        model = model.train()

        train_losses = []
        for train_batch in train_dataloader:
            seq_true = train_batch["time_series"]
            seq_true_labels = train_batch["labels"]
            optimizer.zero_grad()

            seq_true = seq_true.to(device)
            seq_true_labels = seq_true_labels.to(device)
            seq_pred, mu, log_var, label_projection, timeseries_projection = model(seq_true, seq_true_labels)
            reconstruction_loss, kldivergence_loss, similarity_loss = loss_function(seq_true, seq_pred, mu, log_var, label_projection, timeseries_projection, criterion)
            loss = reconstruction_loss + kldivergence_loss + 0*similarity_loss
            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())

        val_losses = []
        model = model.eval()
        with torch.no_grad():
            for val_batch in val_dataloader:
                seq_true = val_batch["time_series"]
                seq_true_labels = val_batch["labels"]

                seq_true = seq_true.to(device)
                seq_true_labels = seq_true_labels.to(device)
                seq_pred, mu, log_var, label_projection, timeseries_projection = model(seq_true, seq_true_labels)

                reconstruction_loss, kld_loss, similarity_loss = loss_function(seq_true, seq_pred, mu, log_var, label_projection, timeseries_projection, criterion)
                val_loss = reconstruction_loss + kld_loss + 0*similarity_loss
                val_losses.append(val_loss.item())

        train_loss = np.mean(train_losses)
        val_loss = np.mean(val_losses)

        history['train'].append(train_loss)
        history['val'].append(val_loss)

        print(f'Epoch {epoch}: train loss {train_loss} val loss {val_loss}')
        plot_results(model, val_dataloader, train_config.n_plots, log_dir, epoch, device)
        if val_loss < best_loss:
            best_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            save_dir = os.path.join(log_dir, 'weights')
            os.makedirs(save_dir, exist_ok=True)
            save_loc = os.path.join(save_dir, 'best_model.pth')
            torch.save(model.state_dict(), save_loc)        

    model.load_state_dict(best_model_wts)
    return model.eval(), history

In [100]:
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")  
foldername = ("./save/sine" + "_" + current_time + "/")

print('model folder:', foldername)
os.makedirs(foldername, exist_ok=True)

num_epochs = cltsp_config.n_epochs
log_dir = os.path.join(foldername, 'results')
os.makedirs(log_dir, exist_ok=True)

model, history = train_model(
    model, 
    train_dataloader, 
    val_dataloader, 
    cltsp_config,
    log_dir,
    device
    )

model folder: ./save/sine_20230810_151623/
Epoch 0: train loss 401.0679082524591 val loss 389.6884031749907
Epoch 1: train loss 398.12937397536837 val loss 390.7028539748419
Epoch 2: train loss 397.16652533180354 val loss 391.3607824416388
Epoch 3: train loss 397.7034780077366 val loss 390.32673209054127
Epoch 4: train loss 395.73431349047723 val loss 391.2378162202381
Epoch 5: train loss 394.75840601155176 val loss 388.0047389439174
Epoch 6: train loss 392.9559528568248 val loss 386.3335447765532
Epoch 7: train loss 391.73895034394735 val loss 388.32173810686385
Epoch 8: train loss 391.63297813296936 val loss 384.1501624697731
Epoch 9: train loss 390.495041209799 val loss 386.4472020467122
Epoch 10: train loss 390.1247392565475 val loss 383.5568578810919
Epoch 11: train loss 389.3116261378471 val loss 382.7824133010138
Epoch 12: train loss 389.1425994714925 val loss 384.953490847633
Epoch 13: train loss 390.593432727873 val loss 386.45772370837983
Epoch 14: train loss 388.894937465845

KeyboardInterrupt: 

In [45]:
import math

class PositionalEncoder(nn.Module):
    def __init__(self, dropout: float=0.1, max_seq_len: int=5000, d_model: int=512, batch_first: bool=True):
        super().__init__()

        self.d_model = d_model
        self.dropout = nn.Dropout(p=dropout)
        self.batch_first = batch_first
        self.x_dim = 1 if batch_first else 0

        position = torch.arange(max_seq_len).unsqueeze(1)        
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_seq_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)
        print(self.pe.shape)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x + self.pe[:x.size(self.x_dim)]
        return self.dropout(x)
    
dim_val = 16
dropout_pos_enc = 0.2 
max_seq_len = 48
n_heads = 8
n_encoder_layers = 4
n_features = 1

positional_encoding_layer = PositionalEncoder(d_model=dim_val, dropout=dropout_pos_enc, max_seq_len=max_seq_len).to(device)
encoder_input_layer = nn.Linear(in_features=n_features, out_features=dim_val).to(device)
encoder_layer = nn.TransformerEncoderLayer(d_model=dim_val, nhead=n_heads, batch_first=True).to(device)
encoder = nn.TransformerEncoder(encoder_layer=encoder_layer, num_layers=n_encoder_layers, norm=None).to(device)


torch.Size([48, 16])


In [46]:
for train_batch in train_dataloader:
    seq_true = train_batch["time_series"]
    seq_true_labels = train_batch["labels"]

    seq_true = seq_true.to(device)
    seq_true_labels = seq_true_labels.to(device)
    src = encoder_input_layer(seq_true) 
    src = positional_encoding_layer(src) # src shape: [batch_size, src length, dim_val] regardless of number of input features
    src = encoder(src=src)
    print(src.shape)


torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])
torch.Size([64, 48, 16])


In [26]:
def time_embedding(pos, d_model=128):
    pe = torch.zeros(pos.shape[0], pos.shape[1], d_model).to(device)
    position = pos.unsqueeze(2)
    div_term = 1 / torch.pow(10000.0, torch.arange(0, d_model, 2).to(device) / d_model)
    pe[:, :, 0::2] = torch.sin(position * div_term)
    pe[:, :, 1::2] = torch.cos(position * div_term)
    return pe

In [30]:
time_embedding(seq_true[:,:,0]).shape

torch.Size([64, 48, 128])

In [41]:
a = torch.zeros((10,2))
a[4] = torch.ones((2,))
a + torch.ones((2,))*0.5

tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [1.5000, 1.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000]])